In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("energydata_complete.csv")
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


**To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.**

In [3]:
data.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [4]:
columns_names= {'date':'Date', 'Appliances':'Appliances', 'lights':'Lights','T1':'T_Kitchen','RH_1':'H_Kitchen','T2':'T_LivingRM', 'RH_2':'H_LivingRM', 'T3':'T_LaundryRM',
       'RH_3':'H_LaundryRM', 'T4':'T_OfficeRM', 'RH_4':'H_OfficeRM', 'T5':'T_BathRM', 'RH_5':'H_BathRM', 'T6':'T_NorthSide', 'RH_6':'H_NorthSide', 'T7':'T_IroningRM', 'RH_7':'H_IroningRm', 'T8':'T_TeensRM',
       'RH_8':'H_TeensRM', 'T9':'T_ParentsRM', 'RH_9':'H_ParentsRM', 'T_out':'T_Outside', 'Press_mm_hg':'Pressure_mm_hg', 'RH_out':'H_Outside', 'Windspeed':'Windspeed','Visibility':'Visibility', 'Tdewpoint':'Tdewpoint', 'rv1':'Random_V1', 'rv2':'Random_V2'}

In [5]:
data.rename(columns = columns_names, inplace= True)

In [6]:
data.columns

Index(['Date', 'Appliances', 'Lights', 'T_Kitchen', 'H_Kitchen', 'T_LivingRM',
       'H_LivingRM', 'T_LaundryRM', 'H_LaundryRM', 'T_OfficeRM', 'H_OfficeRM',
       'T_BathRM', 'H_BathRM', 'T_NorthSide', 'H_NorthSide', 'T_IroningRM',
       'H_IroningRm', 'T_TeensRM', 'H_TeensRM', 'T_ParentsRM', 'H_ParentsRM',
       'T_Outside', 'Pressure_mm_hg', 'H_Outside', 'Windspeed', 'Visibility',
       'Tdewpoint', 'Random_V1', 'Random_V2'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            19735 non-null  object 
 1   Appliances      19735 non-null  int64  
 2   Lights          19735 non-null  int64  
 3   T_Kitchen       19735 non-null  float64
 4   H_Kitchen       19735 non-null  float64
 5   T_LivingRM      19735 non-null  float64
 6   H_LivingRM      19735 non-null  float64
 7   T_LaundryRM     19735 non-null  float64
 8   H_LaundryRM     19735 non-null  float64
 9   T_OfficeRM      19735 non-null  float64
 10  H_OfficeRM      19735 non-null  float64
 11  T_BathRM        19735 non-null  float64
 12  H_BathRM        19735 non-null  float64
 13  T_NorthSide     19735 non-null  float64
 14  H_NorthSide     19735 non-null  float64
 15  T_IroningRM     19735 non-null  float64
 16  H_IroningRm     19735 non-null  float64
 17  T_TeensRM       19735 non-null 

In [8]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [9]:
df = data.drop(columns= ['Date','Lights'])
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
normalised_df.head()

,Appliances,T_Kitchen,H_Kitchen,T_LivingRM,H_LivingRM,T_LaundryRM,H_LaundryRM,T_OfficeRM,H_OfficeRM,T_BathRM,...,T_ParentsRM,H_ParentsRM,T_Outside,Pressure_mm_hg,H_Outside,Windspeed,Visibility,Tdewpoint,Random_V1,Random_V2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [10]:
features_df = normalised_df.drop(columns=['Appliances'])

In [11]:
Appliances_Target = normalised_df['Appliances']

In [12]:
# spliting the dataset into the training and testing dataset. 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, Appliances_Target, test_size=0.3, random_state=42)

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lm = LinearRegression()

In [15]:
#fiting the model to the training dataset
lm_fit = lm.fit(x_train, y_train)
#obtain predictions
predicted_values = lm.predict(x_test)
predicted_values

array([0.03322207, 0.24411599, 0.03400024, ..., 0.06844707, 0.10032325,
       0.05722198])

In [16]:
#MAE Mean Absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3)

0.05

The model might be a good fit with low MAE

In [17]:
#Residual sum of squares
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

45.348

The model might be a good fit with low rss

In [18]:
#Root mean square error
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

The model might be a good fit with low rmse

In [19]:
#Root squared score
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 3)

0.149

r2_score is below 0.5 the model might not be a good fit

In [20]:
normalised_df.columns

Index(['Appliances', 'T_Kitchen', 'H_Kitchen', 'T_LivingRM', 'H_LivingRM',
       'T_LaundryRM', 'H_LaundryRM', 'T_OfficeRM', 'H_OfficeRM', 'T_BathRM',
       'H_BathRM', 'T_NorthSide', 'H_NorthSide', 'T_IroningRM', 'H_IroningRm',
       'T_TeensRM', 'H_TeensRM', 'T_ParentsRM', 'H_ParentsRM', 'T_Outside',
       'Pressure_mm_hg', 'H_Outside', 'Windspeed', 'Visibility', 'Tdewpoint',
       'Random_V1', 'Random_V2'],
      dtype='object')

**Q12**
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [21]:
from sklearn.metrics import r2_score
T2_T6 =normalised_df[['T_LivingRM','T_Outside']]
lm.fit(T2_T6, Appliances_Target )
lm.score(T2_T6, Appliances_Target)


0.014461126905505384

**Q13**
What is the Mean Absolute Error (in two decimal places)?

In [22]:
#MAE Mean Absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

**Q14**
What is the Residual Sum of Squares (in two decimal places)?

In [23]:
#Residual sum of squares
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

**Q15**
What is the Root Mean Squared Error (in three decimal places)?

In [24]:
#Root mean square error
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

**Q16**
What is the Coefficient of Determination (in two decimal places)?

In [25]:
#Root squared score
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

**Q17**
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [26]:
from sklearn.linear_model import LinearRegression
linear_model= LinearRegression()
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df


**Q18**
Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [27]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train,y_train)

Ridge(alpha=0.4)

In [28]:
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

**Q19**
Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [29]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [30]:
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
lasso_weights_df

,Features,Lasso_weight
0,H_Outside,-0.049557
1,H_TeensRM,-0.000110
2,T_Kitchen,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Pressure_mm_hg,-0.000000
6,T_Outside,0.000000
7,H_ParentsRM,-0.000000
8,T_ParentsRM,-0.000000
9,T_TeensRM,0.000000


**Q20**
What is the new RMSE with the Lasso Regression (in 3 decimal places)?

In [31]:
from sklearn import metrics
lasso = Lasso()
lasso.fit(x_train, y_train)
print("Lasso Train RMSE:", np.round(np.sqrt(metrics.mean_squared_error(y_train, lasso.predict(x_train))), 3))
print("Lasso Test RMSE:", np.round(np.sqrt(metrics.mean_squared_error(y_test, lasso.predict(x_test))), 3))

Lasso Train RMSE: 0.096
Lasso Test RMSE: 0.095
